In [1]:
import requests
import pandas as pd
import json

### config
#### API: https://rapidapi.com/apidojo/api/transfermarket
#### https://www.linkedin.com/company/rapidapi/

In [2]:
url = "https://transfermarket.p.rapidapi.com/clubs/list-by-competition"

#	"X-RapidAPI-Key": "<YOUR-RapidAPI-key>",
headers = {
	"X-RapidAPI-Key": "<YOUR-RapidAPI-key>",
	"X-RapidAPI-Host": "transfermarket.p.rapidapi.com"
} 

# seasons to be analised
years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

In [3]:
#function to get retrieved JSON. API data comes with single quota sign instead of double
def cleanSingleQuotas(str):
# some club name cleansing is also required 
    return str.replace("M'gladbach", "Mgladbach").replace("K'lautern", "Klautern").replace("'", '"')

#### Retrieve clubs from current season
#### L1 - German Bundesliga Code

In [50]:
querystring = {"id":"L1","domain":"en"}

response = requests.get(url, headers=headers, params=querystring)

s = response.json()

# API data comes with single quota sign instead of double
j = cleanSingleQuotas(json.dumps(s))

c = json.loads(j)

clubs = pd.json_normalize(c['clubs']).loc[:, 'id':'name']
clubs.to_csv('../data/clubs.csv', index=False)


##### Retrieve the list of players by years

In [ ]:
url = "https://transfermarket.p.rapidapi.com/clubs/get-squad"

clubs = pd.read_csv('../data/clubs.csv')

for y in years:
    print(f'year: {y}')
    players = pd.DataFrame(columns=['id', 'name', 'club', 'joined'])
    for ind in clubs.index:
        querystring = {"id":clubs['id'][ind], "saison_id":y, "domain":"en"}

        response = requests.get(url, headers=headers, params=querystring)

        s = response.json()

        # API data comes with single quota sign instead of double
        j = cleanSingleQuotas(json.dumps(s))

        p = json.loads(j)

        club_players = pd.json_normalize(p['squad']).loc[:, ['id', 'name', 'joined']]
        club_players['club'] = clubs['id'][ind]
        
        players = pd.concat([players, club_players])

    players['year'] = y
    players.to_csv(f'../data/players-{y}.csv', index=False)
